In [64]:
import os
import torch
import numpy as np
import random
from torch.utils.data import DataLoader, Dataset
import math

class SpeedDataset(Dataset):
    def __init__(self, directory, sequence_length=5, augment=True):
        self.data = []
        self.sequence_length = sequence_length
        self.num_classes = (105 - 30) // 10 + 1  # Speed classes from 30-39, ..., 100-105
        self.augment = augment  # Enable or disable augmentation
        self.preprocess_data(directory)
        self.normalize_features()

    def preprocess_data(self, directory):
        for filename in os.listdir(directory):
            if filename.endswith('.txt'):
                speed = float(filename.split('_')[-1].replace('.txt', ''))
                speed_class = 0 if speed < 30 else (int(speed) - 30) // 10
                filepath = os.path.join(directory, filename)
                with open(filepath, 'r') as file:
                    track_data = {}
                    for line in file:
                        points = line.strip().split(',')
                        if len(points) == 6:
                            frame, track_id, x1, y1, x2, y2 = map(float, line.strip().split(','))
                        if len(points) == 7:
                            frame,class_id, track_id, x1, y1, x2, y2 = map(float, line.strip().split(','))
                        if track_id not in track_data:
                            track_data[track_id] = []
                        track_data[track_id].append([x1, y1, x2, y2])

                    for track_id, frames in track_data.items():
                        if len(frames) >= self.sequence_length:
                            features = self.extract_features(frames)
                            overlap = 5  # Overlap for sequence extraction
                            for start_idx in range(0, len(features) - self.sequence_length + 1, self.sequence_length - overlap):
                                end_idx = start_idx + self.sequence_length
                                sequence = features[start_idx:end_idx]
                                self.data.append((sequence, speed_class))

    def extract_features(self, frames):
        features = []
        for i in range(1, len(frames)):
            current_frame = frames[i]
            previous_frame = frames[i-1]
            features.append(self.compute_frame_features(current_frame, previous_frame))
        return features

    def compute_frame_features(self, current_frame, previous_frame):
        x1, y1, x2, y2 = current_frame
        px1, py1, px2, py2 = previous_frame

        width, height = x2 - x1, y2 - y1
        p_width, p_height = px2 - px1, py2 - py1

        x2_change = x2 - px2
        y2_change = y2 - py2
        x1_change = x1 - px1
        y1_change = y1 - py1

        width_change = width - p_width
        height_change = height - p_height
        area_change = (width * height) - (p_width * p_height)
        perimeter_change = (2 * (width + height)) - (2 * (p_width + p_height))

        center_x, center_y = (x1 + x2) / 2, (y1 + y2) / 2
        p_center_x, p_center_y = (px1 + px2) / 2, (py1 + py2) / 2
        center_x_change = center_x - p_center_x
        center_y_change = center_y - p_center_y
        distance_moved = math.sqrt(center_x_change ** 2 + center_y_change ** 2)

        velocity = distance_moved/0.03  # Assuming constant frame rate
        p_velocity = (math.sqrt((px2 - px1) ** 2 + (py2 - py1) ** 2))/0.03
        acceleration = abs(velocity - p_velocity)

        feature_vector = [x1_change, y1_change, x2_change, y2_change,
                          center_x_change, center_y_change, distance_moved, velocity, acceleration]
        return feature_vector

    def normalize_features(self):
        all_features = [feature for sequence, _ in self.data for feature in sequence]
        all_features = np.array(all_features)
        self.mean = np.mean(all_features, axis=0)
        self.std = np.std(all_features, axis=0)
        for i, (sequence, speed_class) in enumerate(self.data):
            normalized_sequence = (sequence - self.mean) / (self.std)
            self.data[i] = (normalized_sequence, speed_class)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputs, output = self.data[idx]
        if self.augment:
            inputs = self.apply_augmentation(inputs)
        return torch.tensor(inputs, dtype=torch.float32), torch.tensor(output, dtype=torch.long)

    def apply_augmentation(self, inputs):
        if random.random() > 0.5:
            inputs = self.add_noise(inputs)
        if random.random() > 0.5:
            inputs = self.time_shift(inputs, shift=random.choice([-1, 1]))
        if random.random() > 0.5:
            inputs = self.scale_features(inputs, scale=random.uniform(0.9, 1.1))
        if random.random() > 0.5:
            inputs = self.mirror_features(inputs)
        return inputs

    def add_noise(self, features, noise_level=0.05):
        noise = np.random.normal(0, noise_level, features.shape)
        return features + noise

    def time_shift(self, features, shift=1):
        if shift > 0:
            return np.vstack([np.zeros((shift, features.shape[1])), features[:-shift]])
        elif shift < 0:
            return np.vstack([features[-shift:], np.zeros((-shift, features.shape[1]))])
        return features

    def scale_features(self, features, scale=1.1):
        return features * scale

    def mirror_features(self, features):
        features_copy = features.copy()
        features_copy[:, [0, 2]] = -features_copy[:, [0, 2]]  # Assume these indices are the x-coordinates
        return features_copy


In [65]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.nn as nn
import torch.nn.functional as F

class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.hidden_dim = hidden_dim
        self.linear = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        weights = F.softmax(self.linear(x), dim=1)
        weighted_output = weights * x
        return weighted_output.sum(1), weights

class SpeedPredictor(nn.Module):
    def __init__(self, sequence_length, feature_size, embedding_dim, hidden_dim, output_size):
        super(SpeedPredictor, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=feature_size, out_channels=embedding_dim, kernel_size=1)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, num_layers=3)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(32, output_size)  
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(in_channels=hidden_dim, out_channels=128, kernel_size=1)
        self.attention = Attention(hidden_dim)
        self.fc1 = nn.Linear(128, 32)

    def forward(self, x):
        x = x.permute(0, 2, 1)  
        x = self.conv1(x)
        x = x.permute(0, 2, 1)  
        x, _ = self.lstm(x)
        x = self.dropout(x)
        x, attn_weights = self.attention(x)
        x = x.unsqueeze(2)
        x = self.conv2(x)
        x = x.squeeze(2)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x  
def init_weights(m):
    if type(m) == nn.LSTM:
        torch.nn.init.xavier_uniform_(m.weight_ih_l0)
        torch.nn.init.orthogonal_(m.weight_hh_l0)
    if type(m) == nn.Linear:
        torch.nn.init.kaiming_normal_(m.weight)

In [66]:
import torch

# Check for CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [67]:

def train(model, train_loader, test_loader, criterion, optimizer, scheduler, epochs):
    for epoch in range(epochs):
        model.train()
        Train_total_loss = 0
        correct_train = 0
        total_train = 0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Ensure inputs and targets are on the same device as model
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()  # First, update the parameters with the current learning rate
            
            Train_total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            correct_train += (predicted == targets).sum().item()
            total_train += targets.size(0)
        
        scheduler.step()  # After optimizer updates, adjust the learning rate

        if epoch % 10 == 0 or epoch == epochs - 1:
            model.eval()
            correct_test = 0
            total_test = 0
            total_loss = 0
            with torch.no_grad():
                for inputs, targets in test_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    total_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    correct_test += (predicted == targets).sum().item()
                    total_test += targets.size(0)
                    
            train_accuracy = 100 * correct_train / total_train
            test_accuracy = 100 * correct_test / total_test
            print(f'Epoch {epoch+1}: Train Loss: {Train_total_loss / len(train_loader)} Test Loss: {total_loss / len(test_loader)}, '
                  f'Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%')


In [69]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import ExponentialLR


train_dataset = SpeedDataset('outputs', sequence_length=10,augment=True)
test_dataset = SpeedDataset('test_outputs', sequence_length=10,augment=False)
print(len(train_dataset))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)
max_classes = train_dataset.num_classes
model = SpeedPredictor(sequence_length=10, feature_size=9, embedding_dim=64, hidden_dim=128, output_size=max_classes)
model.to(device)  
model.apply(init_weights)
criterion = nn.CrossEntropyLoss().to(device)  
optimizer = SGD(model.parameters(), lr=0.01)
scheduler = ExponentialLR(optimizer, gamma=0.9999) 
train(model, train_loader, test_loader, criterion, optimizer, scheduler, epochs=700)


7822
Epoch 1: Train Loss: 2.0598475422177995 Test Loss: 2.049723517149687, Train Accuracy: 18.78%, Test Accuracy: 19.30%
Epoch 11: Train Loss: 1.996635533838856 Test Loss: 2.001786459237337, Train Accuracy: 18.78%, Test Accuracy: 19.30%
Epoch 21: Train Loss: 1.9953655272114033 Test Loss: 1.9989819582551718, Train Accuracy: 18.78%, Test Accuracy: 19.30%
Epoch 31: Train Loss: 1.9918697167416008 Test Loss: 1.9941393472254276, Train Accuracy: 18.78%, Test Accuracy: 19.30%
Epoch 41: Train Loss: 1.9455120500253171 Test Loss: 1.9362765569239855, Train Accuracy: 19.91%, Test Accuracy: 19.15%
Epoch 51: Train Loss: 1.5274024724960327 Test Loss: 1.4718974819406867, Train Accuracy: 33.60%, Test Accuracy: 34.18%
Epoch 61: Train Loss: 1.37519536212999 Test Loss: 1.2724767290055752, Train Accuracy: 42.98%, Test Accuracy: 44.46%
Epoch 71: Train Loss: 1.2717024175488219 Test Loss: 1.2290490625891834, Train Accuracy: 47.29%, Test Accuracy: 44.94%
Epoch 81: Train Loss: 1.2111378438618718 Test Loss: 1.087

In [70]:
from sklearn.metrics import confusion_matrix, classification_report
def print_confusion_matrix_and_report(all_targets, all_preds):
    print(confusion_matrix(all_targets, all_preds))
    print(classification_report(all_targets, all_preds, target_names=[f'Class {30 + i * 10}-{39 + i * 10}' for i in range(max(all_targets) + 1)]))


In [72]:
def evaluate(model, test_loader):
    model.eval()
    correct_test = 0
    total_test = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Ensure inputs and targets are on the same device as model
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            correct_test += (predicted == targets).sum().item()
            total_test += targets.size(0)
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    test_accuracy = 100 * correct_test / total_test
    print("Confusion Matrix and Classification Report:")
    print_confusion_matrix_and_report(all_targets, all_preds)
    return test_accuracy
evaluate(model, test_loader)  

Confusion Matrix and Classification Report:
[[ 842  317   25    1    3    0    0    0]
 [ 154 1013  204   57   35    1    5    0]
 [  13  294  687  197   83    6   10    0]
 [   4   83  223  564  235   16   29    0]
 [   0   11   48  169  636  123   50    0]
 [   2    2    9   38  212  341  128    1]
 [   0    1    1   16   79  106  473   21]
 [   0    0    3    3   16   20  153   59]]
               precision    recall  f1-score   support

  Class 30-39       0.83      0.71      0.76      1188
  Class 40-49       0.59      0.69      0.64      1469
  Class 50-59       0.57      0.53      0.55      1290
  Class 60-69       0.54      0.49      0.51      1154
  Class 70-79       0.49      0.61      0.54      1037
  Class 80-89       0.56      0.47      0.51       733
  Class 90-99       0.56      0.68      0.61       697
Class 100-109       0.73      0.23      0.35       254

     accuracy                           0.59      7822
    macro avg       0.61      0.55      0.56      7822
 wei

59.000255689082074

In [73]:
def evaluate(model, test_loader):
    model.eval()
    correct_test = 0
    total_test = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Ensure inputs and targets are on the same device as model
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            correct_test += (predicted == targets).sum().item()
            total_test += targets.size(0)
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    test_accuracy = 100 * correct_test / total_test
    print("Confusion Matrix and Classification Report:")
    print_confusion_matrix_and_report(all_targets, all_preds)
    return test_accuracy
evaluate(model, test_loader)  

Confusion Matrix and Classification Report:
[[79 17  0  0  0  0  0  0]
 [ 8 77 33  2  2  0  0  0]
 [ 1 24 56 11  3  0  1  0]
 [ 0  6 30 59 25  2  1  0]
 [ 0  0  9 16 44  1  0  0]
 [ 0  1  1  7 14 36 21  0]
 [ 0  0  0  0  0  0  6  7]
 [ 0  0  0  0  1  3 18 10]]
               precision    recall  f1-score   support

  Class 30-39       0.90      0.82      0.86        96
  Class 40-49       0.62      0.63      0.62       122
  Class 50-59       0.43      0.58      0.50        96
  Class 60-69       0.62      0.48      0.54       123
  Class 70-79       0.49      0.63      0.55        70
  Class 80-89       0.86      0.45      0.59        80
  Class 90-99       0.13      0.46      0.20        13
Class 100-109       0.59      0.31      0.41        32

     accuracy                           0.58       632
    macro avg       0.58      0.55      0.53       632
 weighted avg       0.64      0.58      0.59       632



58.06962025316456